In [4]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose



In [5]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [6]:
count=0
timer=0
sum_angle=[]
status='none'
cap = cv2.VideoCapture('KneeBendVideo.mp4')


import datetime
import time
import numpy as np

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)
        current_time = datetime.datetime.now()

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            

            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
 
            angle = calculate_angle(hip, knee, ankle)
            current_time = datetime.datetime.now()
        
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [600,480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            if angle>=140:
                position='down'
            if timer==0 and angle<140:
                timer=current_time.second
            
            if (angle<140) and (timer!=0) and (position!='up'):
                sum_angle.append(angle)
                cv2.putText(image,'timer '+str(current_time.second-timer), [650,50],cv2.FONT_HERSHEY_SIMPLEX, 
                   1, [0,0,0], 2, cv2.LINE_AA)
                if (current_time.second-timer)==8 or (0>(current_time.second-timer)>-52) :
                    count=count+1
                    position='up'
                    if np.mean(sum_angle)>=100:
                        status="bendness"

                    if np.mean(sum_angle)<100:
                        status='stat'
                    sum_angle=[]


            if angle>140 and timer!=0 and 0<current_time.second-timer<8 and (position!='up'):
                cv2.putText(image,'Keep your knee bent' , 
                           tuple(np.multiply(knee, [600,50]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            if angle <=180 and angle>=170:
                timer=0
            cv2.putText(image, 'rep'+ '=' + str(count), [500,50],cv2.FONT_HERSHEY_SIMPLEX, 
                   1, [0,0,0], 2, cv2.LINE_AA)
            cv2.putText(image, 'rep '+str(count)+' status'+'='+str(status), [50,70],cv2.FONT_HERSHEY_SIMPLEX, 
                   1, [0,0,0], 2, cv2.LINE_AA)    
        except:
            pass
        
        
    
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()